In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

In [18]:
test = pd.read_csv('../../input/test.csv',usecols=['id', 'Semana', 'Agencia_ID', 'Canal_ID','Ruta_SAK',
                                                   'Cliente_ID', 'Producto_ID'],
                              index_col  = ['Semana','Cliente_ID','Producto_ID'])                         

In [22]:
test.describe()

,id,Agencia_ID,Canal_ID,Ruta_SAK
count,6.999251e+06,6.999251e+06,6.999251e+06,6.999251e+06
mean,3.499625e+06,2.504463e+03,1.401874e+00,2.138014e+03
std,2.020510e+06,4.010228e+03,1.513404e+00,1.500392e+03
min,0.000000e+00,1.110000e+03,1.000000e+00,1.000000e+00
25%,1.749812e+06,1.311000e+03,1.000000e+00,1.159000e+03
50%,3.499625e+06,1.612000e+03,1.000000e+00,1.305000e+03
75%,5.249438e+06,2.034000e+03,1.000000e+00,2.804000e+03
max,6.999250e+06,2.575900e+04,1.100000e+01,9.950000e+03


In [25]:
groupedTest = test.groupby(level = [0,1,2]).count()

In [20]:
groupedTest[groupedTest['id']>1]

id  Agencia_ID  Canal_ID  Ruta_SAK
Semana Cliente_ID Producto_ID                                    
10     60         34348         2           2         2         2
       12198      1129          2           2         2         2
                  1150          2           2         2         2
       19260      31200         2           2         2         2
       20379      44371         2           2         2         2
       20968      31187         2           2         2         2
       21182      35141         2           2         2         2
                  35144         2           2         2         2
       21581      43038         2           2         2         2
                  43128         2           2         2         2
                  43209         2           2         2         2
       23664      1145          2           2         2         2
                  1212          2           2         2         2
                  40447         2           2         2         2
       24413      31393         2           2         2         2
                  34208         2           2         2         2
                  34255         2           2         2         2
                  43169         2           2         2         2
                  43209         2           2         2         2
       27304      641           2           2         2         2
       28131      43058         2           2         2         2
       28859      1145          2           2         2         2
                  1150          2           2         2         2
       28879      1146          2           2         2         2
       32010      43307         2           2         2         2
       36665      8865          2           2         2         2
       37261      32353         2           2         2         2
                  43185         2           2         2         2
                  43202         2           2         2         2
       48342      31187         2           2         2         2
...                            ..         ...       ...       ...
11     6105744    1109          2           2         2         2
                  1150          2           2         2         2
                  1230          2           2         2         2
                  1232          2           2         2         2
                  1238          2           2         2         2
                  1309          2           2         2         2
                  3144          3           3         3         3
                  3631          2           2         2         2
                  32937         2           2         2         2
                  37058         2           2         2         2
                  43316         2           2         2         2
       6276157    31719         2           2         2         2
       7886715    37577         2           2         2         2
       8083006    35309         2           2         2         2
       8218033    43040         2           2         2         2
       8244009    43202         2           2         2         2
                  43204         2           2         2         2
       8356583    42083         2           2         2         2
       8382138    35977         2           2         2         2
       9484521    1242          2           2         2         2
       9564563    35651         2           2         2         2
       9808903    37005         2           2         2         2
       9838234    30236         2           2         2         2
                  31520         2           2         2         2
                  35972         2           2         2         2
                  35976         2           2         2         2
                  36999         3           3         3         3
       9999999    1242          2           2         2         2
     

In [21]:
test.loc[(11,99999999,2665)]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


id  Agencia_ID  Canal_ID  Ruta_SAK
Semana Cliente_ID Producto_ID                                         
11     99999999   2665         2306934        1129         2         9
                  2665         4894078        1129         2        10

In [34]:
groupedTest2 = test.reset_index().groupby(['Semana','Cliente_ID','Producto_ID','Ruta_SAK']).count()

In [35]:
groupedTest2[groupedTest2['id']>1]

id  Agencia_ID  Canal_ID
Semana Cliente_ID Producto_ID Ruta_SAK                          
10     28859      1145        6636       2           2         2
                  1150        6636       2           2         2
       37261      32353       6611       2           2         2
                  43185       6611       2           2         2
                  43202       6611       2           2         2
       49098      34255       6611       2           2         2
       53721      2025        6640       2           2         2
       81871      1160        6640       2           2         2
       162254     4270        6636       2           2         2
                  5354        6636       2           2         2
       652889     7829        3008       2           2         2
                  30569       3008       2           2         2
                  37058       3007       2           2         2
                  43040       3007       2           2         2
                  45296       3004       2           2         2
       653378     1220        3010       2           2         2
                              3011       2           2         2
                              3013       2           2         2
                              3021       2           2         2
                  1240        3001       6           6         6
                              3002      10          10        10
                              3003       8           8         8
                              3004       6           6         6
                              3005       6           6         6
                              3006       5           5         5
                              3007       6           6         6
                              3008       6           6         6
                              3009       2           2         2
                              3010       3           3         3
                              3011       3           3         3
...                                     ..         ...       ...
11     653378     49769       3011       2           2         2
                              3012       2           2         2
                              3013       2           2         2
                              3016       3           3         3
                              3017       2           2         2
       712603     1232        6620       2           2         2
       920360     2025        6624       2           2         2
       1077133    1240        6627       2           2         2
                  2025        6627       2           2         2
       1118111    1309        1254       2           2         2
                  35144       1254       2           2         2
                  35147       1254       2           2         2
       1226644    1240        6627       2           2         2
       1269372    1238        3202       2           2         2
                  4280        3202       2           2         2
                  37057       3202       2           2         2
       1866330    3270        6620       2           2         2
                  4270        6620       2           2         2
                  31719       6620       2           2         2
       2075614    1112        6625       2           2         2
                  35144       6625       2           2         2
       2388547    40450       6614       2           2         2
       4687485    1250        6614       2           2         2
                  31717       6614       2           2         2
       4712887    43200       900        2           2         2
                  43203       900        2           2         2
       4713082    43307       900        2           2         2
       9484521    1242        6627       2           2         2
       9999999    1242        900        2           2         2
          